In [1]:
# CELL 0 - Mount Drive and install PyG + deps from saved wheels

from google.colab import drive
drive.mount('/content/drive')

WHEEL_DIR = "/content/drive/MyDrive/PyG_wheels_torch29_cu126"
!pip install {WHEEL_DIR}/*.whl

import torch, torch_geometric
print("torch:", torch.__version__)
print("torch_geometric:", torch_geometric.__version__)
from torch_geometric.datasets import QM9
print("QM9 import OK")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/aiohappyeyeballs-2.6.1-py3-none-any.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/aiohttp-3.13.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/aiosignal-1.4.0-py3-none-any.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/attrs-25.4.0-py3-none-any.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/certifi-2025.11.12-py3-none-any.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/charset_normalizer-3.4.4-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/frozenlist-1.8.0-cp312-cp312-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl
Processing ./drive/MyDrive/PyG_wheels_torch29_cu126/fsspec-2025.10.0-py3

In [2]:
# CELL 1 - Clone (or refresh) QM9_project repo

%cd /content
!test -d QM9_project || git clone https://github.com/zeugirdoR/QM9_project.git
%cd QM9_project
!git pull
!ls


/content
Cloning into 'QM9_project'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 6), reused 33 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 21.63 KiB | 21.63 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/QM9_project
Already up to date.
bootstrapwheels.py  how2git.txt			requirements.txt
data		    models			setup_pyg_wheels.py
env		    newColabenv.py		train_qm9_baseline.py
eval_qm9_legacy.py  orig_train_qm9_baseline.py
fastboot.sh	    README.md


In [3]:
# CELL 2 - Warm QM9 test with tiny model (sanity check)

%cd /content/QM9_project

import torch
from torch import nn
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

dataset = QM9(root="data/qm9")
print(f"Total graphs: {len(dataset)}")

n_train = 109_898
n_val   = 13_083
n_test  = 7_850

train_dataset = dataset[:n_train]
val_dataset   = dataset[n_train:n_train + n_val]
test_dataset  = dataset[n_train + n_val:]

print(f"Train/Val/Test = {len(train_dataset)}, {len(val_dataset)}, {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

class WarmTestNet(nn.Module):
    def __init__(self, in_channels, hidden=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1),
        )

    def forward(self, data):
        x = torch.cat([data.x, data.pos], dim=-1)
        batch = data.batch
        num_graphs = int(batch.max()) + 1

        graph_sum = torch.zeros(num_graphs, x.size(-1), device=x.device)
        graph_count = torch.zeros(num_graphs, 1, device=x.device)

        graph_sum.index_add_(0, batch, x)
        graph_count.index_add_(0, batch, torch.ones_like(x[:, :1]))

        graph_mean = graph_sum / graph_count
        out = self.mlp(graph_mean)
        return out.squeeze(-1)

batch = next(iter(train_loader))
in_channels = batch.x.size(-1) + batch.pos.size(-1)
print("In channels (x + pos):", in_channels)

model = WarmTestNet(in_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

batch = batch.to(device)
target_idx = 12  # U0
y = batch.y[:, target_idx]

pred = model(batch)
print("pred shape:", pred.shape, "target shape:", y.shape)

loss = loss_fn(pred, y)
print("Warm test loss:", loss.item())

optimizer.zero_grad()
loss.backward()
optimizer.step()

print("✅ Warm test forward + backward pass completed.")


/content/QM9_project
Using device: cuda


Extracting data/qm9/raw/qm9_v3.zip
Processing...
Using a pre-processed version of the dataset. Please install 'rdkit' to alternatively process the raw data.
Done!


Total graphs: 130831
Train/Val/Test = 109898, 13083, 7850
In channels (x + pos): 14
pred shape: torch.Size([64]) target shape: torch.Size([64])
Warm test loss: 74.15080261230469
✅ Warm test forward + backward pass completed.
